# This is a sample Jupyter Notebook

Below is an example of a code cell.
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

Installs

In [1]:
!pip install -U datasets --quiet
!pip install ipywidgets --quiet
!pip install py7zr --quiet
!pip install transformers --quiet
!pip install torch --quiet
!pip install rouge-score --quiet
!pip install hf_xet --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 11.5 MB/s eta 0:00:00


Importing dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
# Take just a small subset for quick training
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(2000))
small_eval_dataset = dataset["validation"].shuffle(seed=42).select(range(200))

train_dataset = small_train_dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

Tokenizer

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, DatasetDict

raw_datasets = load_dataset("cnn_dailymail", "3.0.0")

# Subsetting BEFORE Tokenization for speed
raw_train = raw_datasets["train"].shuffle(seed=42).select(range(10000))
raw_validation = raw_datasets["validation"].shuffle(seed=42).select(range(1000))

# Convert to DatasetDict for the map function
raw_small_datasets = DatasetDict({"train": raw_train, "validation": raw_validation})

model_id = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)


def preprocess_function(examples):
    # Truncate and tokenize articles (inputs)
    model_inputs = tokenizer(
        examples["article"], max_length=1024, truncation=True, padding="max_length"
    )

    # Truncate and tokenize highlights (targets/labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["highlights"], max_length=128, truncation=True, padding="max_length"
        )

    # Renames 'highlights' target to 'labels'
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the SMALLER dataset using parallel processing
tokenized_datasets = raw_small_datasets.map(
    preprocess_function, batched=True, remove_columns=["article", "highlights", "id"], num_proc=4
)

small_tokenized_train_dataset = tokenized_datasets["train"]
small_tokenized_eval_dataset = tokenized_datasets["validation"]

print(f"Train Dataset size: {len(small_tokenized_train_dataset)} examples")
print(f"Validation Dataset size: {len(small_tokenized_eval_dataset)} examples")
print(f"Train Dataset features: {small_tokenized_train_dataset.features.keys()}")

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

...

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Train Dataset size: 10000 examples
Validation Dataset size: 1000 examples
Train Dataset features: dict_keys(['input_ids', 'attention_mask', 'labels'])


Loading data

In [5]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

# Define train_dataset using the properly tokenized and renamed data
train_dataset = small_tokenized_train_dataset

train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=default_data_collator
)

print(f"DataLoader created with {len(train_dataloader)} batches.")

DataLoader created with 2500 batches.


Tuning

In [ ]:
import torch
from torch.optim import AdamW

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Diagnostic Check
print(f"Device being used: {device}")

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3
print(f"Starting training on {len(train_dataloader)} batches ({len(train_dataset)} examples).")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Use enumerate for tracking progress
    for step, batch in enumerate(train_dataloader):

        # Select only the three necessary keys and move to device
        inputs = {
            "input_ids": batch["input_ids"].to(device),
            "attention_mask": batch["attention_mask"].to(device),
            "labels": batch["labels"].to(device)
        }

        # Forward pass
        outputs = model(**inputs)
        loss = outputs.loss

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # Print loss update every 100 steps
        if (step + 1) % 100 == 0:
            print(f"  Epoch {epoch+1}/{num_epochs}, Step {step+1}/{len(train_dataloader)}, Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_dataloader)
    print(f"\n--- Epoch {epoch+1}/{num_epochs} finished. Average Loss: {avg_loss:.4f} ---")

Device being used: cpu
Starting training on 2500 batches (10000 examples).


Generate Summary

In [ ]:
def generate_summary(article):
    inputs = tokenizer(article, return_tensors="pt", max_length=1024, truncation=True, padding=True)
    inputs = inputs.to(device)

    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=200,
        min_length=50,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Testing
sample_article = """
The annual Galactic Peace Summit, held this year on the orbital station Serenity Prime,
concluded late Tuesday evening with a landmark agreement on interstellar trade routes.
Representatives from the Federation of Planets and the independent merchant guilds spent
three days locked in intense negotiations. The main point of contention was the taxation
of highly valued crystalline resources mined from the Proxima Centauri system's asteroid belt.
The final treaty establishes a zero-tariff policy for all humanitarian aid shipments and a
graduated tax system for commercial goods, which is expected to boost cross-sector economic
activity by 25% in the next fiscal cycle. Dr. Elara Vance, lead negotiator for the Federation,
praised the outcome, calling it "a new era of cooperation and shared prosperity." The agreement
is set to be ratified by all major planetary councils within the next two weeks and will take
effect immediately thereafter. Security forces reported no incidents, making this one of the
smoothest summits in history.
"""

summary = generate_summary(sample_article)
print(f"--- Article ---")
print(f"{sample_article.strip()}")
print(f"\n--- Generated Summary ---")
print(f"{summary}")

Rouge score

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def evaluate_summary(reference, generated):
    scores = scorer.score(reference, generated)
    return scores

# Example evaluation
reference_summary = dataset['test'][0]['highlights']
generated_summary = generate_summary(sample_article)

scores = evaluate_summary(reference_summary, generated_summary)
print(scores)